<h1>Test des modèles : Oussama FIKRI, Brahim GOUALI et Hamza BOUKHRISSE</h1>

**Référence** : Les étiquettes catégoriques correspondantes aux étiquettes numériques dans l'ensemble de données sur le cancer de la peau sont les suivantes :
<ol> <li>akiec</li>
    <li>bcc</li> 
    <li>bkl</li> 
    <li>df</li>  
    <li>nv</li> 
    <li>vasc</li> 
    <li>mel</li>
</ol>

In [12]:
import numpy as np
from PIL import Image
import tensorflow as tf

# Load and preprocess a single test image
image_path = "/kaggle/input/basedir/base_dir/val_dir/akiec/ISIC_0032404.jpg"
img = Image.open(image_path)
img = img.resize((28, 28))  # Resize the image to 28x28 pixels
img_array = np.array(img)
preprocessed_img = img_array.reshape((1, 28, 28, 3))  # Reshape to match the expected input shape

# Load the TFLite model
model_path = '/kaggle/input/pretrainedmodels1/Skin.tflite'
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Set the input tensor
input_data = np.array(preprocessed_img, dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run inference
interpreter.invoke()

# Get the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

# Perform the prediction
predicted_class = np.argmax(output_data)
print("Predicted class:", predicted_class)

Predicted class: 0


In [11]:
import numpy as np
from PIL import Image
from tensorflow import keras
model = keras.models.load_model('/kaggle/input/predtrainedmodel2/Skin Cancer.h5')

# Load and preprocess a single test image
image_path = "/kaggle/input/basedir/base_dir/val_dir/akiec/ISIC_0032404.jpg"
img = Image.open(image_path)
img = img.resize((28, 28))  # Resize the image to 28x28 pixels
img_array = np.array(img)
preprocessed_img = img_array.reshape((1, 28, 28, 3))  # Reshape to match the expected input shape

# Load the model

# Perform the prediction
predictions = model.predict(preprocessed_img)
predicted_class = np.argmax(predictions)
print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 272ms/step
Predicted class: 0


In [9]:
import numpy as np
from PIL import Image
from tensorflow import keras
model = keras.models.load_model('/kaggle/input/pretrainedmodels1/trained_model.h5')

# Load and preprocess a single test image
image_path = "/kaggle/input/basedir/base_dir/val_dir/akiec/ISIC_0032404.jpg"
img = Image.open(image_path)
img = img.resize((28, 28))  # Resize the image to 28x28 pixels
img_array = np.array(img)
preprocessed_img = img_array.reshape((1, 28, 28, 3))  # Reshape to match the expected input shape

# Load the model

# Perform the prediction
predictions = model.predict(preprocessed_img)
predicted_class = np.argmax(predictions)
print("Predicted class:", predicted_class)

1/1 [==============================] - 0s 209ms/step
Predicted class: 0


# Utilisation des trois modèles dans la classification

In [13]:
import numpy as np
from PIL import Image
from tensorflow import keras
import tensorflow as tf

# Load and preprocess a single test image
image_path = "/kaggle/input/basedir/base_dir/val_dir/akiec/ISIC_0032404.jpg"
img = Image.open(image_path)
img = img.resize((28, 28))  # Resize the image to 28x28 pixels
img_array = np.array(img)
preprocessed_img = img_array.reshape((1, 28, 28, 3))  # Reshape to match the expected input shape

# Load the models
model1 = keras.models.load_model('/kaggle/input/predtrainedmodel2/Skin Cancer.h5')
model2 = keras.models.load_model('/kaggle/input/pretrainedmodels1/trained_model.h5')
model3_path = '/kaggle/input/pretrainedmodels1/Skin.tflite'
interpreter = tf.lite.Interpreter(model_path=model3_path)
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Perform predictions for each model
predictions = []
predictions.append(np.argmax(model1.predict(preprocessed_img)))
predictions.append(np.argmax(model2.predict(preprocessed_img)))

# Set the input tensor for model3
input_data = np.array(preprocessed_img, dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run inference for model3
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
predictions.append(np.argmax(output_data))

# Perform voting
final_prediction = np.argmax(np.bincount(predictions))
print("Final predicted class:", final_prediction)

1/1 [==============================] - 0s 202ms/step
Final predicted class: 0


### Validation de l'approche adoptée

In [23]:
import os
import numpy as np
from PIL import Image
from tensorflow import keras
import tensorflow as tf

# Set the paths to the models and validation directory
model1_path = '/kaggle/input/predtrainedmodel2/Skin Cancer.h5'
model2_path = '/kaggle/input/pretrainedmodels1/trained_model.h5'
model3_path = '/kaggle/input/pretrainedmodels1/Skin.tflite'
validation_dir = "/kaggle/input/basedir/base_dir/val_dir/"
classes = {'nv':4,
           'mel':6,
           'bkl':2, 
           'bcc':1,
           'vasc':5,
           'akiec':0,
           'df':3}
# Load the models
model1 = keras.models.load_model(model1_path)
model2 = keras.models.load_model(model2_path)
interpreter = tf.lite.Interpreter(model_path=model3_path)
interpreter.allocate_tensors()

# Get input and output details for model3
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Initialize variables for accuracy calculation
total_samples = 0
correct_predictions = 0

# Iterate over the subdirectories in the validation directory
for class_name in os.listdir(validation_dir):
    class_dir = os.path.join(validation_dir, class_name)
    
    # Iterate over the images in each class directory
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        
        # Load and preprocess the image
        img = Image.open(image_path)
        img = img.resize((28, 28))
        img_array = np.array(img)
        preprocessed_img = img_array.reshape((1, 28, 28, 3))
        
        # Perform predictions for each model
        predictions = []
        predictions.append(np.argmax(model1.predict(preprocessed_img)))
        predictions.append(np.argmax(model2.predict(preprocessed_img)))
        
        # Set the input tensor for model3
        input_data = np.array(preprocessed_img, dtype=np.float32)
        interpreter.set_tensor(input_details[0]['index'], input_data)
        
        # Run inference for model3
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        predictions.append(np.argmax(output_data))
        
        # Perform voting
        final_prediction = np.argmax(np.bincount(predictions))
        # Check if the final prediction matches the ground truth class
        if final_prediction == int(classes[class_name]):
            correct_predictions += 1
        total_samples += 1

# Calculate accuracy
accuracy = correct_predictions / total_samples
print("Accuracy:", accuracy)

1/1 [==============================] - 0s 20ms/step
Accuracy: 0.9936034115138592
